### Bibliotecas utilizadas

In [2]:
import io
import boto3
import gspread
import pandas as pd
from gspread_dataframe import set_with_dataframe

### Lendo CSV do S3

In [ ]:
s3_client = boto3.client('s3')

res = s3_client.get_object(Bucket="trusted", Key="dados.csv")

csv_content = res['Body'].read()

### Leitura e Tratamento  de dados

In [5]:
# df = pd.read_csv(io.BytesIO(csv_content))
df = pd.read_parquet("./dados.parquet")

df = df.dropna()

df['FP minimo'] = ''
df.loc[df['sensor_model'] == 'Fluke 1735', 'FP minimo'] = 92
df.loc[df['sensor_model'] == 'Fluke 1735','data'] *= 100

df['Horario'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%H:%M')
df['Mês'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%m')


df['created_at'] = df['created_at'].astype('datetime64[ns]').dt.strftime('%d/%m/%Y %H:%M')
df.rename(columns={
'sensor_model':'Origem', 'measure_unit':'Unidade de Medida', 'device':'Dispositivo', 'location':'Localizacao', 'data_type':'Tipo de Dado', 'data':'Registro', 'created_at':'Data/Hora'
}, inplace=True)

df.to_csv('dados_tratados.csv', index=False, encoding='utf-8')

### Exportação para Google Sheets

In [28]:
gc = gspread.service_account(filename='credentials.json')
spreadsheet = gc.open("wc-grupo-10")
print([ws.title for ws in spreadsheet.worksheets()])
worksheet = spreadsheet.worksheet("wc")
set_with_dataframe(worksheet, df)
print("DataFrame enviado para o Google Sheets com sucesso.")

['wc', 'wordcloud-data', 'energia total', 'Fator de potência', 'consumo total de energia', 'corrente elétrica', 'analise emocional', 'Detalhe1-alface-bom']
DataFrame enviado para o Google Sheets com sucesso.
